In [3]:
import numpy as np
import cv2
import pickle
from tensorflow.keras.models import load_model
import pyttsx3
import threading
import time

In [5]:
#############################################
# CAMERA SETTINGS
frameWidth = 640        # Camera width
frameHeight = 480       # Camera height
brightness = 180        # Brightness level

threshold = 0.75        # Prediction probability threshold
font = cv2.FONT_HERSHEY_SIMPLEX
prediction_interval = 5  # Make predictions every 'n' frames
confidence_threshold = 0.85  # Higher confidence threshold for voice output
no_sign_class = 0  # Assuming class '0' is for "no sign" or background
#############################################

# SETUP VIDEO CAMERA
cap = cv2.VideoCapture(0)
cap.set(3, frameWidth)     # Width
cap.set(4, frameHeight)    # Height
cap.set(10, brightness)    # Brightness

# LOAD THE TRAINED MODEL
model = load_model("model.h5")  # Ensure 'model.h5' is in the same directory

# IMAGE PREPROCESSING FUNCTIONS
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def equalize(img):
    return cv2.equalizeHist(img)

def preprocessing(img):
    img = grayscale(img)
    img = equalize(img)
    img = img / 255  # Normalize pixel values
    return img

# CLASS NAMES FOR PREDICTIONS
def getClassName(classNo):
    classes = [
        'Speed Limit 20 km/h', 'Speed Limit 30 km/h', 'Speed Limit 50 km/h',
        'Speed Limit 60 km/h', 'Speed Limit 70 km/h', 'Speed Limit 80 km/h',
        'End of Speed Limit 80 km/h', 'Speed Limit 100 km/h',
        'Speed Limit 120 km/h', 'No passing',
        'No passing for vehicles over 3.5 metric tons',
        'Right-of-way at the next intersection', 'Priority road', 'Yield', 'Stop',
        'No vehicles', 'Vehicles over 3.5 metric tons prohibited', 'No entry',
        'General caution', 'Dangerous curve to the left', 'Dangerous curve to the right',
        'Double curve', 'Bumpy road', 'Slippery road', 'Road narrows on the right',
        'Road work', 'Traffic signals', 'Pedestrians', 'Children crossing',
        'Bicycles crossing', 'Beware of ice/snow', 'Wild animals crossing',
        'End of all speed and passing limits', 'Turn right ahead', 'Turn left ahead',
        'Ahead only', 'Go straight or right', 'Go straight or left', 'Keep right',
        'Keep left', 'Roundabout mandatory', 'End of no passing',
        'End of no passing by vehicles over 3.5 metric tons'
    ]
    return classes[classNo] if 0 <= classNo < len(classes) else "Unknown"

# INITIALIZE TEXT-TO-SPEECH ENGINE
engine = pyttsx3.init()
engine.setProperty('rate', 150)  # Speed of speech
engine.setProperty('volume', 1)  # Volume level

def speak(predicted_class):
    engine.say(f"The sign is: {predicted_class}")
    engine.runAndWait()

# MAIN LOOP FOR PREDICTION
frame_counter = 0  # To track frames and limit predictions

while True:
    # READ FRAME FROM CAMERA
    success, imgOriginal = cap.read()

    if not success:
        print("Failed to grab frame from webcam.")
        break

    # IMAGE PROCESSING
    img = cv2.resize(imgOriginal, (32, 32))
    img = preprocessing(img)
    img = img.reshape(1, 32, 32, 1)  # Reshape for model input

    # PREDICT IMAGE every 'prediction_interval' frames
    if frame_counter % prediction_interval == 0:
        predictions = model.predict(img)
        classIndex = np.argmax(predictions, axis=1)[0]
        probabilityValue = np.amax(predictions)

        # Only process predictions with confidence above the threshold
        if probabilityValue > threshold:
            # Handle "no sign" case
            if classIndex == no_sign_class:
                className = "No sign detected"
            else:
                className = getClassName(classIndex)

            # Only speak if the confidence is above the threshold
            if probabilityValue >= confidence_threshold:
                # Speak the predicted class in a separate thread to avoid lag
                threading.Thread(target=speak, args=(className,)).start()

            # Display the class name and probability on the screen
            cv2.putText(imgOriginal, f"CLASS: {className}", (20, 35), font, 0.75, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(imgOriginal, f"PROBABILITY: {round(probabilityValue * 100, 2)}%", (20, 75), font, 0.75, (0, 255, 0), 2, cv2.LINE_AA)

    frame_counter += 1

    # SHOW THE WEBCAM OUTPUT
    cv2.imshow("Result", imgOriginal)

    # PRESS 'q' TO EXIT
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# RELEASE CAMERA AND CLOSE WINDOWS
cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━